In [87]:
import pandas as pd
import json
from tqdm import tqdm

In [88]:
data_fname = "./Relation_extraction_herb_chemical.jsonl"

In [89]:
all_data = []

In [90]:
f = open(data_fname, "r")
line = f.readline()

In [91]:
j_data = json.loads(line)

In [92]:
relations = "relations"
head_span = "head_span"
child_span = "child_span"
label = "label"
start = "start"
end = "end"
text = "text"

In [93]:
j_data["relations"][0]

{'head': 82,
 'child': 87,
 'head_span': {'start': 337,
  'end': 349,
  'token_start': 82,
  'token_end': 82,
  'label': 'CHEMICAL'},
 'child_span': {'start': 375,
  'end': 392,
  'token_start': 86,
  'token_end': 87,
  'label': 'HERB'},
 'color': '#c5bdf4',
 'label': 'POSITIVE'}

# Data Formatting

In [94]:
data = (j_data[text], j_data[text][j_data[relations][0][head_span][start]:j_data[relations][0][head_span][end]], j_data[relations][0][label])

In [95]:
data

('), was given to the rabbits and the blood samples were withdrawn at different time intervals (0, 0.5, 1, 1.5, 2, 3, 4, 6, 8, 12, 24 and 36 h) from marginal ear vein after dosing and theophylline in plasma was analyzed by hplc method. results: it was observed that there a significant differences in the cmax, auc, aumc, t1/2, and mrt of theophylline when coadministered with commiphora myrrha which indicate that the herb affect the metabolism and elimination when coadministered with theophylline . conclusion: our results suggested that concurrent use of investigated herb alters the pharmacokinetics of theophylline.',
 'theophylline',
 'POSITIVE')

In [102]:
f.seek(0)
all_data_dict = {i: json.loads(j_data) for i, j_data in enumerate(f)}
all_data_dict[0]

{'text': '), was given to the rabbits and the blood samples were withdrawn at different time intervals (0, 0.5, 1, 1.5, 2, 3, 4, 6, 8, 12, 24 and 36 h) from marginal ear vein after dosing and theophylline in plasma was analyzed by hplc method. results: it was observed that there a significant differences in the cmax, auc, aumc, t1/2, and mrt of theophylline when coadministered with commiphora myrrha which indicate that the herb affect the metabolism and elimination when coadministered with theophylline . conclusion: our results suggested that concurrent use of investigated herb alters the pharmacokinetics of theophylline.',
 '_input_hash': 447283788,
 '_task_hash': -565715972,
 'spans': [{'start': 182,
   'end': 194,
   'token_start': 50,
   'token_end': 50,
   'label': 'CHEMICAL'},
  {'start': 337,
   'end': 349,
   'token_start': 82,
   'token_end': 82,
   'label': 'CHEMICAL'},
  {'start': 375,
   'end': 392,
   'token_start': 86,
   'token_end': 87,
   'label': 'HERB'},
  {'start': 

# Preparing all Data

In [11]:
def get_mention(text: str, span: dict) -> str:
  return text[span[start]: span[end]]

In [109]:
from tqdm import tqdm
f.seek(0)
all_data = []
for line in tqdm(f):
  j_data = json.loads(line)
  curr_text = j_data[text]
  for relation in j_data[relations]:
    entity_mention_1 = get_mention(curr_text, relation[head_span])
    entity_mention_2 = get_mention(curr_text, relation[child_span])
    all_data.append((curr_text, entity_mention_1, relation[head_span][start], relation[head_span][end], relation[head_span][label], entity_mention_2, relation[child_span][start], relation[child_span][end], relation[child_span][label], relation[label]))

165it [00:00, 7945.13it/s]


In [110]:
total_data = len(all_data)
total_data

433

In [114]:
all_corefs = [(tup[1], tup[5]) for tup in all_data if tup[9] == "COREF"]

In [116]:
all_corefs

[('herb', 'commiphora myrrha'),
 ('danggui', 'their'),
 ('honghua', 'their'),
 ('gfj', 'grapefruit'),
 ('ginkgo biloba', 'gbe'),
 ('tanshinone', 'tan'),
 ('policosanol', 'each herbal medicine'),
 ('echinacea', 'each herbal medicine'),
 ('cytochrome p450', 'drug')]

In [117]:
corefs = [
    "herb",
    "their",
    "gfj",
    "gbe",
    "tan",
    "each herbal medicine",
    "drug"
]

Removing unwanted data

In [118]:
# Index 9 is reletion type
all_data = [tup for tup in all_data if tup[9] != "COREF"]
total_data = len(all_data)
total_data

424

In [119]:
# Index 4 and 8 is entity labels for head and child
all_data = [tup for tup in all_data if tup[4] != tup[8]]
total_data = len(all_data)
total_data

415

In [120]:
# Index 4 and 8 is entity labels for head and child
all_data = [tup for tup in all_data if not ((tup[4] == "HERB_COMPOUND" and tup[8] == "HERB") or (tup[8] == "HERB_COMPOUND" and tup[4] == "HERB"))]
total_data = len(all_data)
total_data

415

In [121]:
valid_size = int(len(all_data) * .1)
valid_size

41

In [122]:
train_data = all_data[:total_data - valid_size]
len(train_data)

374

In [123]:
valid_data = all_data[total_data - valid_size:]
len(valid_data)

41

In [124]:
train = train_data[40]

curr_text = train[0]
entity_mention_1 = train[1]
entity_mention_1_start = train[2]
entity_mention_1_end = train[3]
#entity_mention_1_label = train[4]
entity_mention_2 = train[5]
entity_mention_2_start = train[6]
entity_mention_2_end = train[7]
# entity_mention_2_label = train[8]
curr_label = train[9]
curr_text = curr_text[:entity_mention_1_start:] + f"<e1> {entity_mention_1} </e1>" + curr_text[entity_mention_1_end:]


if entity_mention_2_start > entity_mention_1_start:
    entity_mention_2_start +=  + len("<e1> ") + len(" </e1>")
    entity_mention_2_end += entity_mention_2_start + len(entity_mention_2)



curr_text = curr_text[:entity_mention_2_start:] + f" <e2> {entity_mention_2} </e2> " + curr_text[entity_mention_2_end:]

first_entity_start = min(entity_mention_1_start, entity_mention_2_start, entity_mention_1_end, entity_mention_2_end)
last_entity_end = max(entity_mention_1_start, entity_mention_2_start, entity_mention_1_end, entity_mention_2_end)

sentence_start = curr_text.rfind(".", 0, first_entity_start)
sentence_end = curr_text.find(".", last_entity_end, -1)

if sentence_start == -1:
    sentence_start = 0

if sentence_end == -1:
    sentence_end = len(curr_text)

curr_text[sentence_start:sentence_end]

'. emblica) and terminalia bellerica (<e1> t. bellerica </e1>) extracts on human hepatocellular carcinoma (hepg2), and lung carcinoma (a549) cells and their synergistic effect with doxorubicin or  <e2> cisplatin </e2> '

In [165]:
new_train = []
tags = ["<e1>", "</e1>", "<e2>", "</e2>"]
for train in tqdm(train_data):
  curr_text = train[0]
  entity_mention_1 = train[1]
  entity_mention_1_start = train[2]
  entity_mention_1_end = train[3]
  #entity_mention_1_label = train[4]
  entity_mention_2 = train[5]
  entity_mention_2_start = train[6]
  entity_mention_2_end = train[7]
  #entity_mention_2_label = train[8]
  curr_label = train[9]


  if(entity_mention_1 in corefs or entity_mention_2 in corefs):
    print(f"{entity_mention_1} {entity_mention_2} skipped coref")
    continue

  if len(entity_mention_1) > len(entity_mention_2):
    s_new = curr_text.replace(entity_mention_1, '<e1> ' + entity_mention_1 + ' </e1>')
    s_new = s_new.replace(entity_mention_2, '<e2> ' + entity_mention_2 + ' </e2>')
  else:
    s_new = curr_text.replace(entity_mention_2, '<e2> ' + entity_mention_2 + ' </e2>')
    s_new = s_new.replace(entity_mention_1, '<e1> ' + entity_mention_1 + ' </e1>')

  first_entity_start = min(entity_mention_1_start, entity_mention_2_start)
  last_entity_end = max(entity_mention_1_end, entity_mention_2_end)

  sentence_start = s_new.rfind(".", 0, first_entity_start)
  sentence_end = s_new.find(".", last_entity_end)

  if sentence_start == -1:
    sentence_start = 0

  if sentence_end == -1:
      sentence_end = len(curr_text)

  skip = False
  for tag in tags:
    if tag not in s_new[sentence_start:sentence_end]:
      skip = True
      break
  if not skip:
    new_train.append((s_new[sentence_start:sentence_end], entity_mention_1, entity_mention_2, curr_label))

100%|██████████| 374/374 [00:00<00:00, 93590.46it/s]

herb theophylline skipped coref
their clopidogrel skipped coref
gfj atorvastatin skipped coref
gbe cytochrome p450 skipped coref
gbe cyp skipped coref
tan cyp skipped coref
cyp1a1 gbe skipped coref
1a2 gbe skipped coref
2b1 gbe skipped coref
cyp1a1 tan skipped coref
1a2 tan skipped coref
2b1 tan skipped coref
hops drug skipped coref
humulus lupulus l. drug skipped coref
hop drug skipped coref
hop drug skipped coref
atorvastatin gfj skipped coref


In [166]:
new_train[10]

('aim: <e1> fruit </e1> juice reduces the plasma concentrations of several <e2> β-adrenoceptor blockers </e2>, likely by inhibiting oatp2b1-mediated intestinal absorption',
 'fruit',
 'β-adrenoceptor blockers',
 'POSITIVE')

In [167]:
train_df = pd.DataFrame(new_train, columns=["sentences", "entity_1_mention", "entity_2_mention",  "type"])
train_df

,sentences,entity_1_mention,entity_2_mention,type
0,. results: it was observed that there a signif...,theophylline,commiphora myrrha,POSITIVE
1,. results: it was observed that there a signif...,commiphora myrrha,theophylline,POSITIVE
2,. results: as compared with <e1> sesame </e1> ...,sesame,glibenclamide,POSITIVE
3,.05 vs <e2> glibenclamide </e2> monotherapy) a...,sesame,glibenclamide,POSITIVE
4,after stopping treatment with <e1> st john's w...,st john's wort,cyclosporine,POSITIVE
...,...,...,...,...
333,. conclusion: the radix bupleuri and <e1> vine...,vinegar-baked radix bupleuri,cyp2c19,POSITIVE
334,.05) from that of controlled rats. conclusion:...,radix bupleuri,cyp2c19,POSITIVE
335,.05) from that of controlled rats. conclusion:...,radix bupleuri,cyp2c9,POSITIVE
336,the pharmacokinetics of <e1> caffeine </e1> in...,caffeine,radix bupleuri,POSITIVE


In [168]:
valid_df = pd.DataFrame(valid_data, columns=["sentences", "entity_1_mention", "start1", "end1", "entity_1_label", "entity_2_mention", "start2", "end2", "entity_2_label", "type"])
valid_df.drop(["start1", "start2", "end1", "end2"], axis=1, inplace=True)
valid_df

,sentences,entity_1_mention,entity_1_label,entity_2_mention,entity_2_label,type
0,the pharmacokinetics of tolbutamide in all vin...,tolbutamide,CHEMICAL,vinegar-baked radix bupleuri,HERB,NEGATIVE
1,the pharmacokinetics of tolbutamide in all vin...,radix bupleuri,HERB,tolbutamide,CHEMICAL,POSITIVE
2,the plasma concentrations of the six probes we...,midazolam,CHEMICAL,radix bupleuri,HERB,POSITIVE
3,the plasma concentrations of the six probes we...,midazolam,CHEMICAL,vinegar-baked radix bupleuri,HERB,POSITIVE
4,the plasma concentrations of the six probes we...,dextromethorphan,CHEMICAL,radix bupleuri,HERB,POSITIVE
5,the plasma concentrations of the six probes we...,dextromethorphan,CHEMICAL,vinegar-baked radix bupleuri,HERB,POSITIVE
6,the plasma concentrations of the six probes we...,chlorzoxazone,CHEMICAL,radix bupleuri,HERB,POSITIVE
7,the plasma concentrations of the six probes we...,chlorzoxazone,CHEMICAL,vinegar-baked radix bupleuri,HERB,POSITIVE
8,the reason of different therapeutic effects of...,radix bupleuri,HERB,cyp2c9,CHEMICAL,POSITIVE
9,the reason of different therapeutic effects of...,vinegar-baked radix bupleuri,HERB,cyp2c19,CHEMICAL,POSITIVE


In [169]:
from sklearn import model_selection
def randomKFold_func(df, n_splits=5):
    # creat a new colunm called kfold and fill it with -1
    df['kfold'] = -1
    # randomize the rows of the data
    df = df.sample(frac=1).reset_index(drop=True)
    # initiate the kfold class for model selection module
    kf = model_selection.KFold(n_splits=n_splits)
    # fill the new kfold column
    for f, (t_, v_) in enumerate(kf.split(X=df)):
        df.loc[v_, 'kfold'] = f 
    return df

dataset_df_kfold = randomKFold_func(train_df, n_splits=5)

In [170]:
dataset_df_kfold

,sentences,entity_1_mention,entity_2_mention,type,kfold
0,it is unclear whether some component of the <e...,ginseng,digoxin,SPECULATIVE,0
1,in vitro studies from our laboratory suggest t...,kaempferol,erythromycin,POSITIVE,0
2,<e1> digoxin </e1> therapy was maintained at a...,digoxin,ginseng,POSITIVE,0
3,".32) for <e1> ginkgo </e1> treatment, and 1.00...",ginkgo,cyp2c9,NEGATIVE,0
4,other herbal remedies with the potential to mo...,ginseng,cytochrome p450,POSITIVE,0
...,...,...,...,...,...
333,interactions of several commonly used herbal m...,ginko biloba,imatinib,POSITIVE,4
334,by taking advantage of the high protein bindin...,chan su,digoxin,POSITIVE,4
335,interactions of several commonly used herbal m...,st. john's wort,tacrolimus,POSITIVE,4
336,our results clarified that oral administration...,a. officinarum,indomethacin,POSITIVE,4


In [163]:
path_save ="./datasets/"

In [164]:
dataset_df_kfold.to_csv(path_save + "train_new.csv")
valid_df.to_csv(path_save + "valid_new.csv")